In [1]:
"""
This file used to solve the question2 
"""
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import linprog
from scipy.interpolate import interp1d

N = 40 # 节点个数
M = 62 # 街道条数


## ## 代码说明：
    本代码以泉州市丰泽区的部分区域为例，图片见群文件“泉州市丰泽区.jpg”，
    该部分共计:
   - （拐角处）40个
   - 边（街道）62条
   - 一个方形（重要区域）

In [2]:
"""
   使用 邻接矩阵 E 确定，各个节点之间的连接情况。
   定义每两个节点之间的连线为一条街道。
   
   邻接矩阵E的每一个元素e，实际上都是代表一条街道。
   
   每一个e都有三个要素：
       to:    街道的一个端点。
       from:  街道的另一个端点。
       length:街道的长度。
    
    i就是e所在的列数，j就是e所在的行数。
    length由实际情况确定：
        如果 length 为 0: to、from之间没有街道
        如果 length > 0, to、from之间有街道且街道的长度为E[i][j]
        
"""
columns = np.arange(1, 41, 1)
index = np.arange(1, 41, 1)
data = np.zeros([40, 40])


class Edge(object):
    def __init__(self, _from, _to, length, mode):
        self.to = _to
        self.from_ = _from
        self.length = length
        self.mode = mode
        
 
X = pd.DataFrame(data, columns=columns, index=index)
E = []


def link(a, b, d):
    X.iat[a - 1, b - 1] = d
    X.iat[b - 1, a - 1] = d
    e = Edge(a, b, d, 1);
    E.append(e)


step = 2

link(1, 2, 5 * step)
link(1, 5, 5 * step)

link(2, 3, 4 * step)
link(2, 6, 5 * step)

link(3, 4, 1 * step)
link(3, 7, 5 * step)

link(5, 6, 6 * step)
link(5, 9, 5 * step)
link(5, 13, 10 * step)

link(6, 7, 4 * step)
link(6, 10, 5 * step)

link(7, 8, 1 * step)
link(7, 11, 4 * step)

link(8, 12, 4 * step)

link(9, 10, 6 * step)
link(9, 16, 10 * step)

link(10, 11, 4 * step)
link(10, 14, 4 * step)

link(11, 12, 1 * step)
link(11, 15, 4 * step)

link(12, 14, 7 * step)
link(12, 19, 10 * step)

link(13, 14, 1 * step)
link(13, 16, 6 * step)

link(14, 15, 5 * step)
link(14, 17, 5 * step)

link(15, 18, 4 * step)

link(16, 24, 10 * step)
link(16, 17, 5 * step)

link(17, 20, 3 * step)
link(17, 18, 5 * step)

link(18, 19, 2 * step)
link(18, 21, 2 * step)

link(19, 22, 2 * step)

link(20, 25, 3 * step)
link(20, 21, 4 * step)

link(21, 22, 1 * step)
link(21, 23, 1 * step)

link(22, 26, 3 * step)

link(24, 27, 0.5 * step)
link(24, 25, 8 * step)

link(25, 28, 1 * step)
link(25, 26, 8 * step)

link(26, 29, 0.5 * step)

link(27, 30, 0.5 * step)
link(27, 28, 7 * step)

link(28, 29, 8 * step)
link(29, 32, 0.5 * step)

link(30, 31, 8 * step)
link(30, 35, 12 * step)

link(31, 32, 7 * step)
link(31, 33, 3 * step)

link(32, 34, 5 * step)

link(33, 34, 8 * step)
link(33, 36, 4 * step)

link(34, 37, 3 * step)

link(35, 38, 6 * step)

link(36, 39, 6 * step)
link(36, 37, 8 * step)

link(37, 40, 6 * step)

link(38, 39, 7 * step)
link(39, 40, 7 * step)

X[X == 0] = None
C = X
X

,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
1,NaN,10.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.0,NaN,8.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.0,NaN,2.0,NaN,NaN,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10.0,NaN,NaN,NaN,NaN,12.0,NaN,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,10.0,NaN,NaN,12.0,NaN,8.0,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,10.0,NaN,NaN,8.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
ext = 1 # 离散化常量
# class SubSection(object):
#     def __init__(self, father, safe_value, d, mode):
#         self.father = father
#         self.safe_value = safe_value
#         self.d = d
#         self.mode = mode
    

# class Section(object):
#     def __init__(self, safe_value=0, from_list=[], to_list=[], mode=1):
#         self.son = []
#         self.safe_value = safe_value
#         self.from_list = from_list
#         self.to_list = to_list
#         self.mode=mode
    
    
#     def insert(self, safe_value=0, d, mode=0):
#         sub_section = SubSection(self, 0, )
        
        
# # TODO: 插入子节点

# for i in range(len(E)):
#     edge = E[i]
#     length = edge.length
    
# TODO: 配置子节def push()点
subV = [[0 for _ in range(N)] for _ in range(N)]
for i in range(N):
    for j in range(N):
        if X.iat[i, j] > 0:
            subV[i][j] = [0 for _ in range(np.int(X.iat[i, j]))]
subV = pd.DataFrame(subV,index=np.arange(1, 41), columns=np.arange(1, 41))
subV[subV == 0] = None

subC = np.array(subV)
subC = subC.tolist()

print(subV)
subV = np.array(subV).tolist()

                                1                               2   \
1                             None  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   
2   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]                            None   
3                             None        [0, 0, 0, 0, 0, 0, 0, 0]   
4                             None                            None   
5   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]                            None   
6                             None  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   
7                             None                            None   
8                             None                            None   
9                             None                            None   
10                            None                            None   
11                            None                            None   
12                            None                            None   
13                            None                            None   
14                  

In [9]:
"""
   shi ta

"""
def randomON(low, high):
    return np.random.randint(low, high + 1)
    
cost_data = [randomON(0, 2) for _ in range(N)]
    

for i in range(N):
    for j in range(N):
        if subC[i][j] is not None:
            for z in range(len(subC[i][j])):
                subC[i][j][z] = randomON(0, 1)

           
# C = np.array(C).tolist()
# for i in range(N):
#     for j in range(N):
#         if C[i][j] > 0:
#             C[i][j] = randomON(0, 2)
#         else:
#             C[i][j] = None
# C

V = np.zeros(40)
C = np.array(cost_data)
C

array([1, 1, 1, 2, 1, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 0, 0, 0, 2, 2, 0, 1,
       0, 1, 2, 0, 1, 0, 0, 0, 1, 2, 2, 1, 2, 2, 1, 0, 2, 0])

In [15]:
# def getSafeValue(distance, x, y):
#     if C[x][y] is not None:
#         return 1 / (1 + math.exp(distance)) + getSubSafeValue(0, x, y)
#     else:
#         return 0
    

# def getSubSafeValue(distance, x, y):
#     res = 0
#     if subC[x][y] is not None:
#         for z in range(len(subC[x][y])):
#             if subC[x][y][z] == 1:
#                 res += 1 / (1 + math.exp(abs(z - distance)))
#     return res


def getSafeValue(distance):
        return 1 / ( 1 + math.exp(distance) )

    
def addSafeValue(x):
    if C[x] > 0:
        V[x] += 1
        for y in range(N):
            if X.iat[x, y] > 0:
                distance = X.iat[x, y]
                V[y] += getSafeValue(int(distance))
                for z in range(int(distance)):
                    subV[x][y][z] += getSafeValue(z + 1)

for i in range(N):
    addSafeValue(i)
subV

[[None,
  [2.344707106849976,
   1.5960146101105879,
   0.2371293658878339,
   0.0899310498104578,
   0.03346425462142428,
   1.0123631157831738,
   1.004555255972003,
   1.0016767506523319,
   0.0006169728799311587,
   1.0002269893435125],
  None,
  None,
  [2.344707106849976,
   1.5960146101105879,
   1.2371293658878337,
   1.0899310498104577,
   1.0334642546214248,
   1.0123631157831738,
   0.004555255972003227,
   1.0016767506523319,
   1.000616972879931,
   0.00022698934351217197],
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [[0.8068242641099853,
   1.3576087660663527,
   0.14227761953270035,
   1.0539586298862746,
   0.020078552772854565,
   1.0074178694699043,
   0.0027331535832019363,
   1.0010060503913991,
   1.0003701837279586,
   0.000136193

In [ ]:
# """
#   定义每一个点的安全值S，显然每一个点的安全值至于其所在的街道和所在街道的位置有关。
#   我们将每一条街道的点离散化，通过安全函数计算出街道的 端点to 和 端点from 对这点
#   安全值的影响s1, s2，某一点的安全值s = s1 + s2
# """

# # 该返回安全值的模型未定义，这里就直接返回 1 / (1 + e^ r)
# def getSafeValue(distance, position):
#     if C[position] > 0:
#         return 1 / ( 1 + math.exp(distance) )
#     else:
#         return 0;

# safe_value = []
# # print(safe_value)

# # TODO:离散化街道
# for i in range(len(E)):
#     e = E[i]
#     length = e.length
#     to = e.to
#     from_ = e.from_
#     print("from:", from_,"to:", to, "length", length)
#     value_list = []
#     for j in range(1, int(length) + 1):
#         value_list.append(getSafeValue(j, from_) + getSafeValue(length - j, to))
#     safe_value.append(value_list)
    
    

# safe_value_exl = pd.DataFrame(safe_value)
# safe_value_exl



In [ ]:
# """
# 以
# 第二条街道的安全系数分布曲线
# 为例：
# """
# index = [2, 3, 6, 7]

# fig = plt.figure()

# def draw_figure(p, i):
    
#     pos = int('22' + str(p))
    
#     e = E[i]
#     length = e.length
#     x = np.arange(1, length + 1)
#     y = safe_value[i]
    
    
#     ax = fig.add_subplot(pos)
#     ax.set_xticks(range(1, 1 + length))
    
#     xnew = np.linspace(x.min(), x.max(), 1000)

#     f = interp1d(x, y, kind='cubic')
    
#     ynew = f(xnew)

#     #plt.plot(x, y, 'o', xnew, f(xnew))
    
#     plt.scatter(xnew, ynew, s=1, cmap='rainbow', c=ynew)

# for i in range(len(index)):
#     draw_figure(i + 1, index[i])

# plt.savefig("examples.png")